In [4]:
##setting configuration
conf=sc.getConf()
conf.set("spark.executor.memory","4g")
conf.getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.executor.memory', '4g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '192.168.1.104'),
 ('spark.app.id', 'local-1520678716176'),
 ('spark.history.fs.logDirectory', 'file:///C:/tmp/spark-events'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.driver.port', '49684')]

In [ ]:
import collections
lines = sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/ml-100k/ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))

In [ ]:
def parse(line):
    splits=line.split(',')
    return (int(splits[2]),int(splits[3]))
rdd=sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/fakefriends.csv")
ageFriends=rdd.map(parse)
ageFriends=ageFriends.mapValues(lambda x:(x,1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))
ageFriends=ageFriends.mapValues(lambda x:x[0]/x[1])
##heavy
ageFriends=ageFriends.sortByKey()
ageFriends.collect()

In [ ]:
def parseForTemp(line):
    splits=line.split(",")
    return (splits[0],splits[1],float(splits[3]) * 0.1 * (9.0 / 5.0) + 32.0)
lines=sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/1800.csv")
stationTempDay=lines.filter(lambda x:",TMIN," in x).map(parseForTemp)
stationTemp=stationTempDay.map(lambda x:(x[0],x[2]))
minimumStationTemp=stationTemp.reduceByKey(lambda x,y:min(x,y))
mins=minimumStationTemp.collect()
print(mins)
##maximum temp
MaxstationTempDay=lines.filter(lambda x:",TMAX," in x).map(parseForTemp)
MaxstationTemp=MaxstationTempDay.map(lambda x:(x[0],x[2]))
MaximumStationTemp=MaxstationTemp.reduceByKey(lambda x,y:max(x,y))
maxs=MaximumStationTemp.collect()
print(maxs)

In [ ]:
import re
def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())

rdd=sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/Book.txt")
words=rdd.flatMap(normalizeWords)
counts=words.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
counts=counts.map(lambda x:(x[1],x[0])).sortByKey(ascending= False)
counts.collect()

In [ ]:
def parseCustData(line):
    splits=line.split(",")
    return (int(splits[0]),float(splits[2]))
custData=sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/customer-orders.csv")
custRdd=custData.map(parseCustData)
custAggr=custRdd.reduceByKey(lambda x,y:x+y)
data=custAggr.sortBy(lambda x:x[1],ascending=True).collect()
for item in data:
    print("customer Id-"+str(item[0])+";Total amount-"+str(item[1]))

In [ ]:
import collections
def sortKey(elem):
    return elem[1]

def loadMovieNames():
    movieNames = {}
    with open("E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/ml-100k/ml-100k/u.item") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])]=fields[1]
    return movieNames

movieNames_bc=sc.broadcast(loadMovieNames())
lines = sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/ml-100k/ml-100k/u.data")
ratings = lines.map(lambda x: movieNames_bc.value[int(x.split()[1])])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items(),key=sortKey))
for key, value in sortedResults.items():
    print("movie %s has been watched %i times" % (key, value))

In [ ]:
##Most popular superhero
def loadHeroNames():
    movieNames = {}
    with open("E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/Marvel-Names.txt") as f:
        for line in f:
            fields = line.split('\"')
            movieNames[int(fields[0])]=fields[1]
    return movieNames

def friendsPerComic(line):
    splits=line.split()
    return (int(splits[0]),len(splits)-1)
    
marvelSHNames=sc.broadcast(loadHeroNames())
marvelRdd=sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/Marvel-Graph.txt")
marvelFriends=marvelRdd.map(friendsPerComic)
marvelTotalFriends=marvelFriends.map(lambda x:(marvelSHNames.value[x[0]],x[1])).reduceByKey(lambda x,y:x+y)
marvelTotalFriendsSorted=marvelTotalFriends.sortBy(lambda x:x[1],ascending=False)
marvelTotalFriendsSorted.collect()

In [2]:
##To improve results-Pearson correlation coefficient,Jaccard coefficient,conditional probablity
## Adjust threshold, Use genre info
import sys
from pyspark import SparkConf, SparkContext
from math import sqrt

def loadMovieNames():
    movieNames = {}
    with open("E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/ml-100k/ml-100k/u.item", encoding='ascii', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

#Python 3 doesn't let you pass around unpacked tuples,
#so we explicitly extract the ratings now.
def makePairs( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))

def filterDuplicates( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2

def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

print("\nLoading movie names...")
nameDict = loadMovieNames()

data = sc.textFile("file:///E:/Kapil/software-study/Big_Data_NoSql/spark/spark-course/ml-100k/ml-100k/u.data")

# Map ratings to key / value pairs: user ID => movie ID, rating
ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

# Emit every movie rated together by the same user.
# Self-join to find every combination.
joinedRatings = ratings.join(ratings)

# At this point our RDD consists of userID => ((movieID, rating), (movieID, rating))

# Filter out duplicate pairs
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)

# Now key by (movie1, movie2) pairs.
moviePairs = uniqueJoinedRatings.map(makePairs)

# We now have (movie1, movie2) => (rating1, rating2)
# Now collect all ratings for each movie pair and compute similarity
moviePairRatings = moviePairs.groupByKey()

# We now have (movie1, movie2) = > (rating1, rating2), (rating1, rating2) ...
# Can now compute similarities.
moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()

# Save the results if desired
#moviePairSimilarities.sortByKey()
#moviePairSimilarities.saveAsTextFile("movie-sims")

# Extract similarities for the movie we care about that are "good".
if (len(sys.argv) > 1):

    scoreThreshold = 0.97
    coOccurenceThreshold = 50

    movieID = 50

    # Filter for movies with this sim that are "good" as defined by
    # our quality thresholds above
    filteredResults = moviePairSimilarities.filter(lambda pairSim: \
        (pairSim[0][0] == movieID or pairSim[0][1] == movieID) \
        and pairSim[1][0] > scoreThreshold and pairSim[1][1] > coOccurenceThreshold)

    # Sort by quality score.
    results = filteredResults.map(lambda pairSim: (pairSim[1], pairSim[0])).sortByKey(ascending = False).take(10)

    print("Top 10 similar movies for " + nameDict[movieID])
    for result in results:
        (sim, pair) = result
        # Display the similarity result that isn't the movie we're looking at
        similarMovieID = pair[0]
        if (similarMovieID == movieID):
            similarMovieID = pair[1]
        print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))


Loading movie names...
Top 10 similar movies for Star Wars (1977)
Empire Strikes Back, The (1980)	score: 0.9895522078385338	strength: 345
Return of the Jedi (1983)	score: 0.9857230861253026	strength: 480
Raiders of the Lost Ark (1981)	score: 0.981760098872619	strength: 380
20,000 Leagues Under the Sea (1954)	score: 0.9789385605497993	strength: 68
12 Angry Men (1957)	score: 0.9776576120448436	strength: 109
Close Shave, A (1995)	score: 0.9775948291054827	strength: 92
African Queen, The (1951)	score: 0.9764692222674887	strength: 138
Sting, The (1973)	score: 0.9751512937740359	strength: 204
Wrong Trousers, The (1993)	score: 0.9748681355460885	strength: 103
Wallace & Gromit: The Best of Aardman Animation (1996)	score: 0.9741816128302572	strength: 58


In [3]:
sqlContext.clearCache()